# Implementing Basic Neural Networks with LENGTH

This is the first practical exercise of our course [Applied Edge AI](https://learn.ki-campus.org/courses/edgeai-hpi2022).
In this exercise, you are going to implement a few basic functions and library components of neural networks from scratch in Python.

Before we can actually start, install our pip package, which installs the surrounding library LENGTH (Lightning-fast Extensible Neural-network Guarding The HPI), by running the following cell (click the triangular "Play" button next to the cell or in the top bar).

In [1]:
!pip install length_hpi
# if you get the warning "Failed to establish a new connection", go to the side bar on the right, then "Settings" and switch on "Internet"
# you can safely ignore errors such as "ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. [...]"
# since we only need our length package, Pillow, and numpy in this exercise

Now we are going to prepare a few imports for the following code cells:

In [2]:
import numpy as np

import length.functions as F

import length.tests.optimizer_tests.test_sgd as sgd_tests
import length.tests.layer_tests.test_fully_connected as fc_tests
import length.tests.function_tests.test_mean_squared_error as mse_tests
import length.tests.function_tests.test_relu as relu_tests

from length import constants
from length.data_sets import MNIST, FashionMNIST
from length.function import Function
from length.models import MLP
from length.optimizer import Optimizer

# Task 1: SGD

In one of the lectures in our course we learned about SGD.
In the following task we want to compute the parameter delta to actually implement SGD.
If you look up the formula on our course slides, the `param_deltas` are subtracted by our framework, thus we do *not* need to multiply our result with -1 in the code.
Also note, that the variable `gradients` is the *list* of computed derivatives, thus the derivative part of the formula is already computed here.

In [3]:
class SGD(Optimizer):
    """
    An optimizer that does plain Stochastic Gradient Descent
    (https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Iterative_method)
    :param lr: the learning rate to use for optimization
    """

    def __init__(self, lr):
        self.learning_rate = lr

    def run_update_rule(self, gradients, _):
        # :param gradients: a list of all computed gradient arrays
        # :return: a list of deltas for each provided gradient

        # TODO: implement SGD update rule and store the result in a variable called param_deltas (as a list)
        # HINT: it can be solved in a single line with a list comprehension ;)
        # TASK START - Start coding here:
        param_deltas = [self.learning_rate*grad for grad in gradients]
        return param_deltas

If you want to test your solution, you can execute the following code cell.
If your solution passes our test, it will simply print `Test passed.`
If it does not work, you will get an error.
In this case you need to fix the code above.
(And do not forget to run the above code cell again!)

In [4]:
sgd_tests.SGD = SGD
sgd_tests.test_sgd()
print("Test passed.")

# Task 2: Fully Connected Layer

It seems we have not learned a lot about a "fully connected" layer (sometimes also called a "Dense" layer) in our lecture so far.
However we learned about Perceptrons and Multi Layer Perceptrons (MLPs), and fully connected layers are the building block of these early models for machine learning.

They simply store a weight between each possible input and output value and a bias for each output value.
For example, if we have 2 inputs $i_0,i_1$ and 3 outputs $o_0,o_1,o_2$, we store 6 weight values $w_{00}, w_{01}, w_{10}, w_{11}, w_{20},w_{21}$, one value for each pair of one input and one output, and three bias values $b_0,b_1,b_2$, one for each output.
Then during the forward pass the outputs of a fully connected layer are calculated as
$$o_x = \sum_{y=0}^1{(i_y \cdot w_{xy}) + b_x} \text{ for } 0 \leq x < 3.$$
This is simplified for a single element but in a neural network we work with mini-batches (processing a small number of samples at the same time).
In this case, we can use the [dot product](https://numpy.org/doc/stable/reference/generated/numpy.dot.html) of two arrays to simplify this element-wise multiplication.

Here is a small very simple code snippet for the forward pass of our example case above with a batch size of 5.
The batch axis complicates matters slightly, but pay attention to the shape of each array and how the output size changes.
You can play around with different input and output sizes here if you want:

In [5]:
batch_size = 5
num_inputs = 2
num_outputs = 3

i = np.arange(batch_size * num_inputs).reshape(batch_size, num_inputs)
w = np.arange(num_outputs * num_inputs).reshape(num_outputs, num_inputs)
b = np.zeros(num_outputs)
print(f"Inputs {i.shape}:\n", i)
print(f"Weights {w.shape}:\n", w)
print(f"Bias {b.shape}:\n", b)

output = np.dot(i, w.T) + b
print(f"Output {output.shape}:\n", output)

#### One interesting fact here can help you with implementing the backward pass:
applying the dot product on two arrays with shapes (5,2) and (2,3) "removes" the common axis with size 2 and results in an array of (5,3).
You can use this knowledge to figure out which arrays need to be transposed to get the correct shape during the *backward* pass.

Furthermore, you can recap how multiplication (between weights and inputs) affects the gradients in our lecture video [Computational Graph](https://learn.ki-campus.org/courses/edgeai-hpi2022/items/3btmrU8Ds8rVDk8SEUz1pU).
(Remember instead of using simple multiplication we can use the dot product for arrays.)
In the same video you can also recap what happens when we add *two* values in a computational graph (in this case the result of the dot product and our bias), so you can later on implement the *backward* pass for the *bias* correctly.

Also you may have noted, that we *transposed* the weight array in the example above - "swapping" the axes from (3,2) to (2,3) - with numpy before the dot product by calling `.T`.

This is because we recommend storing the weight array in a *transposed* way in your implementation (similar to our example above).

One final hint before you are ready to start implementing: we use the variable name `x` for the inputs (`grad_x` for the gradients with respect to the inputs) in our code below, instead of `i` in the example above (we only used `i` above so it maches the previous formula).

For the neuron (without activation function):

$$
\mathbf{y}_{(o, bs)} = \mathbf{W}_{(i,o)}^T \cdot \mathbf{x}_{(i,bs)} + \mathbf{b}_{(o,bs)} 
$$

Where $i$, $o$, and $bs$ are the number of inputs, outputs and batch elements, respectively.
The derivatives of $\mathbf{y}$ related to the variables ot its function are,

$$
\frac{\partial \mathbf{y}}{\partial \mathbf{x}} = \mathbf{W}_{(i,o)}
$$

$$
\frac{\partial \mathbf{y}}{\partial \mathbf{W}} = \mathbf{x}_{(i, bs)}
$$

$$
\frac{\partial \mathbf{y}}{\partial \mathbf{b}} = \left\{ 1 \right\}_{(o, bs)}
$$

For the backward pass, we use the chain rule. Hence, being $\mathbf{output}_{(o, bs)}$ the vector of output of the model, we have the gradient applied to the layer in analysis

$$
\mathbf{grad}_x = \frac{\partial \mathbf{output}}{\partial \mathbf{x}}_{(i, bs)} = \frac{\partial \mathbf{y}}{\partial \mathbf{x}}_{(i, o)} \mathbf{grad}_{(o, bs)}
$$

$$
\mathbf{grad}_W = \frac{\partial \mathbf{output}}{\partial \mathbf{W}}_{(i, o)} = \frac{\partial \mathbf{y}}{\partial \mathbf{W}}_{(i, bs)} \mathbf{grad}_{(o, bs)}^T
$$

$$
\mathbf{grad}_b = \frac{\partial \mathbf{output}}{\partial \mathbf{b}}_{(o, bs)} = \mathbf{grad}_{(o, bs)}
$$

Where $\mathbf{grad}$ is the gradient obtained by applying the chain rule on the higher layers.
**Observation** Since the bias is defined for just one sample, the $\mathbf{grad}_b$ is reduced to the mean related to the 0 axis (that is, over the batch sample).


In [6]:
from length.layer import Layer
from length.constants import DTYPE
from length.initializers.xavier import Xavier


class FullyConnected(Layer):
    """
    The FullyConnected Layer is one of the base building blocks of a neural network. It computes a weighted sum
    over the input, using a weight matrix. It furthermore applies a bias term to this weighted sum to allow linear
    shifts of the computed values.
    """
    name = "FullyConnected"

    def __init__(self, num_inputs, num_outputs, weight_init=Xavier()):
        super().__init__()

        # TODO: initialize our weights with correct shape, using the weight initializer 'weight_init'
        # here are two hints:
        # 1. store an array of zeros in `self._weights` with the correct shape (we recommend storing it transposed as in our simple example above) and use dtype=DTYPE
        # 2. call `weight_init` with our freshly created array `self._weights` to initialize the array properly
        self._weights = np.zeros(shape=(num_outputs, num_inputs,), dtype=DTYPE)
        weight_init(self._weights)
        
        # TODO: initialize `self.bias` with an array of zeros in the correct shape and use dtype=DTYPE
        self.bias = np.zeros(shape=(num_outputs,), dtype=DTYPE)
        
    @property
    def weights(self):
        # Transform weights between internal and external representation
        return self._weights.T

    @weights.setter
    def weights(self, value):
        # Transform weights between internal and external representation
        self._weights = value.T

    def internal_forward(self, inputs):
        x, = inputs
        
        # TODO: calculate the output of this layer and store it in a variable `result`
        #       (hint: you can look at our simple example above)
        result = np.dot(x, self._weights.T) + self.bias
        
        return result,

    def internal_backward(self, inputs, gradients):
        x, = inputs
        grad_in, = gradients
        
        # TODO: calculate gradients with respect to inputs for this layer
        # 1. calculate and store gradients for (the batch of) the inputs `x` in `grad_x`
        #    (hint: instead of simple multiplication we need to use the dot product for arrays)
        # 2. calculate and store gradients for the weights `w` in `grad_w`
        #    (hint: the shapes of `grad_w` and `self._weights` must be equal, so try to figure out which axes is "removed" by applying the dot product)
        # 3. calculate and store gradients for the bias `b` in `grad_b`
        #    (hint: gradients from multiple sources in the computational graph need to be added up)
        grad_x = np.dot(grad_in, self._weights)
        grad_w = np.dot(grad_in.T, x)
        grad_b = np.sum(grad_in, axis=0)
        
        assert grad_x.shape == x.shape
        assert grad_w.shape == self._weights.shape
        assert grad_b.shape == self.bias.shape

        return grad_x, grad_w, grad_b

    def internal_update(self, parameter_deltas):
        delta_w, delta_b = parameter_deltas
        
        # TODO: apply updates to weights (self._weights) and bias (self.bias) according to deltas from optimizer
        # if you remember our instructions on how to implement SGD, we said: "[...] the param_deltas are subtracted by our framework [...]"
        # so this is all we need to do here.
        self._weights = self._weights - delta_w 
        self.bias = self.bias - delta_b

If you want to test your solution, you can execute the following code cell.
If your solution passes our test, it will simply print `Test passed.`
If it does not work, you will get an error.
In this case you need to fix the code above.
(And do not forget to run the above code cell again!)

In [7]:
fc_tests.FullyConnected = FullyConnected
fc_tests.test_initialization()
fc_tests.test_fully_connected_forward()
fc_tests.test_fully_connected_backward()
print("Test passed.")

# Task 3: Mean Squared Error

To train a model, we also need a loss function.
These loss functions mathematically define how our model should be optimized (and thus learn to solve a certain task).

A very simple loss function, which still can be effective in training models is the [Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error).
Therefore in the following task, we are going to implement this function.
The corresponding [wikipedia article](https://en.wikipedia.org/wiki/Mean_squared_error) should explain everything you need to know, if you are not yet familiar with the function.

In [8]:
class MeanSquaredError(Function):
    """
    This function calculates the Mean Squared Error between two given vectors, as described in
    https://en.wikipedia.org/wiki/Mean_squared_error
    """
    name = "MeanSquaredError"

    def __init__(self):
        super().__init__()
        # TODO: add more initialization if necessary
        self.error = None

    @staticmethod
    def create_one_hot(data, shape):
        assert len(shape) == 2, "Providing integers as second input to MSE only works with two dimensional input vectors"
        # TODO: create a one-hot representation out of the given label vector (with dtype=DTYPE)
        # Example: assume `data` is [2, 3, 0], and the desired `shape` is (3, 4)
        #          in this case we have 4 possible classes and 3 samples, belonging to class 2, class 3, and class 0 
        #          therefore we need to set a 1 at position 2, 3, 0 for each sample respectively
        #          the resulting vector should look like this:
        #          result = [[0, 0, 1, 0], [0, 0, 0, 1], [1, 0, 0, 0]]
        # Hint: initialize an array of zeros with the given `shape`, set 1s where needed and in the end return your created array
        result = np.zeros(shape, dtype=DTYPE)
        for i in range(len(data)):
            result[i, data[i]] = 1
            
        return result
        
    def internal_forward(self, inputs):
        x1, x2 = inputs

        if np.issubdtype(x2.dtype, np.integer):
            x2 = self.create_one_hot(x2, x1.shape)

        self.error = np.mean((x2 - x1)**2).astype(DTYPE)
        
        return self.error,

    def internal_backward(self, inputs, gradients):
        x1, x2 = inputs
        gx, = gradients
        
        # TODO: calculate the gradients of this function with respect to its (two) inputs
        # (hint: the derivative depends on the inputs (here you could use an intermediate 
        #        result from the forward pass) and the size of the inputs)
        
        
        if np.issubdtype(x2.dtype, np.integer):
            # in case we used MSE as loss function, we won't propagate any gradients to the loss
            x2 = self.create_one_hot(x2, x1.shape)
            gradient_1 = 2/x1.size * -1 *(x2 - x1) * gx
            
            return gradient_1, None
        
        gradient_1 = 2/x1.size * -1 *(x2 - x1) * gx
        gradient_2 = 2/x2.size * (x2 - x1) * gx

        return gradient_1, gradient_2


def mean_squared_error(input_1, input_2):
    """
    This function calculates the Mean Squared Error between input_1 and input_2. Both inputs should be vectors of the
    same shape. You can also supply a one-dimensional list of integers.
    If you do so this vector will be converted to a one_hot representation that fits to the shape of the second
    input
    :param input_1: the first vector of any shape
    :param input_2: the second vector. Needs to have the same shape as the first vector, or be a one-dimensional int vector
    :return: the mean squared error between input_1 and input_2
    """
    return MeanSquaredError()(input_1, input_2)

If you want to test your solution, you can execute the following code cell.
If your solution passes our test, it will simply print `Test passed.`
If it does not work, you will get an error.
In this case you need to fix the code above.
(And do not forget to run the above code cell again!)

In [9]:
mse_tests.MeanSquaredError = MeanSquaredError
mse_tests.mean_squared_error = mean_squared_error
mse_tests.test_mean_squared_error_forward_zero_loss()
mse_tests.test_mean_squared_error_forward_loss()
mse_tests.test_mean_squared_error_forward_int_input()
mse_tests.test_mean_squared_error_backward()
mse_tests.test_mean_squared_error_backward_with_label()
print("Test passed.")

# Task 4: ReLU (Rectified Linear Unit)

In our course we learned about how the ReLU function helped to solve the problem of vanishing gradients in the video [A Concise History of Neural Networks (3/4)](https://learn.ki-campus.org/courses/edgeai-hpi2022/items/22nlBMim7pwAX8A7gTI7qn).

In the next task, we are going to implement this function by filling in the missing forward and backward pass.

In [10]:
class Relu(Function):
    """
    The Relu Layer is a non-linear activation
    """
    name = "ReLU"

    def __init__(self):
        super().__init__()
        # TODO: add more initialization of class member variables if necessary
        self.output = None
        
    def internal_forward(self, inputs):
        x, = inputs
        # TODO: calculate forward pass of ReLU function and store it in `activated_inputs`
        # (we can store any variables we need for the calculation of the backward pass in a class member variable here as well)
        x_gt_zero = x > 0 
        relu_x = x.copy()
        relu_x[~x_gt_zero] = 0
        self.output = relu_x 
        return self.output, 
        
    def internal_backward(self, inputs, gradients):
        x, = inputs
        grad_in, = gradients
        
        # TODO: calculate gradients of ReLU function with respect to the input and store it in grad_x
        # you can first calculate the derivative of ReLU itself (hint: it depends on the input of the forward pass)
        # and then use element-wise multiplication of the calculated derivative with the `grad_in` gradients
        relu_back = self.output.copy()
        o_gt_zero = relu_back > 0
        relu_back[o_gt_zero] = 1
        grad_x = np.multiply(grad_in, relu_back)
        assert grad_x.shape == x.shape
        return grad_x,


def relu(x):
    """
    This function computes the element-wise ReLU activation function (https://en.wikipedia.org/wiki/Rectifier_(neural_networks))
    on a given input vector x.
    :param x: the input vector
    :return: a rectified version of the input vector
    """
    return Relu()(x)

If you want to test your solution, you can execute the following code cell.
If your solution passes our test, it will simply print `Test passed.`
If it does not work, you will get an error.
In this case you need to fix the code above.
(And do not forget to run the above code cell again!)

In [11]:
relu_tests.relu = relu
relu_tests.Relu = Relu
relu_tests.test_relu_forward()
relu_tests.test_relu_backward()
print("Test passed.")

# Building a MultiLayerPerceptron (MLP)

Now we have implemented all the building blocks required to build and train a MultiLayerPerceptron.
Please explore the code below a bit, especially how our layers are initialized in the `__init__` function, and how they are used during the forward pass.
Also you can see how our functions (`relu` and `mean_squared_error`) are used, but do not need initialization or saving, since they - in contrast to our `FullyConnected` layers - store no weights .

We only need to save the output of our loss function in `self.loss` to be able to calculate gradients in the backward pass.

In [12]:
class ActivatedMLP:
    
    def __init__(self):
        self.fully_connected_1 = FullyConnected(784, 512)
        self.fully_connected_2 = FullyConnected(512, 512)
        self.fully_connected_3 = FullyConnected(512, 10)

        self.loss = None
        self.predictions = None
    
    def forward(self, batch, train=True):
        hidden = relu(self.fully_connected_1(batch.data))
        hidden = relu(self.fully_connected_2(hidden))
        self.predictions = self.fully_connected_3(hidden)
        self.loss = mean_squared_error(self.predictions, batch.labels)
        
    def backward(self, optimizer):
        self.loss.backward(optimizer)
        

# Train Loop

Now we still need a training loop implementation.
We have provided one below, but you should carefully try to understand the concepts of what is happening here.

(Next week, we are going to implement our own training loop with PyTorch in the exercise. It is going to look slightly different.)

In [13]:
def train_loop(args, data_set, model, optimizer):
    for epoch in range(args.num_epochs):
        # train our model
        for iteration, batch in enumerate(data_set.train):
            model.forward(batch)
            model.backward(optimizer)

            if iteration % args.train_verbosity == 0:
                accuracy = F.accuracy(model.predictions, batch.labels).data
                print("train: epoch: {: 2d}, loss: {: 5.2f}, accuracy {:.2f}, iteration: {: 4d}".
                      format(epoch, model.loss.data, accuracy, iteration), end="\r")
        
        # test our model
        print("\nrunning test set...")
        sum_accuracy = 0.0
        sum_loss = 0.0
        for iterations, batch in enumerate(data_set.test):
            model.forward(batch, train=False)
            sum_accuracy += F.accuracy(model.predictions, batch.labels).data
            sum_loss += model.loss.data
        nr_batches = iterations - 1
        print(" test: epoch: {: 2d}, loss: {: 5.2f}, accuracy {:.2f}".
              format(epoch, sum_loss / nr_batches, sum_accuracy / nr_batches))
        
        # a very simple update rule for the learning rate, reduce it by a factor of 10 after the first and the fourth epoch
        # do *not* change this if you want to solve our bonus task (otherwise it will change the expected results)
        if epoch % 3 == 0:
            optimizer.learning_rate *= 0.1


# Running the Training

Finally, we can train our MLP.
We have provided code to load two datasets [MNIST](http://yann.lecun.com/exdb/mnist/) and [FasionMNIST](https://github.com/zalandoresearch/fashion-mnist).
The first one classifies handwritten digits in grayscale images, the latter one classifies items of clothing based on grayscale images.

Since these datasets are extremely small, we can even train our model on CPU and it should not take longer than a few minutes.
You should be able to simply run the code below right now.
Note how the train and test accuracy changes during the training process.

Training with the default settings (using our implemented SGD optimizer starting with a learning rate of 0.1) our MLP on MNIST should achieve about 0.78 (78%) **test** accuracy after 5 epochs of training.

If you implement the Adam optimizer in the bonus task at the end of this notebook, you need to edit the code below to enable training with it, look for the instructions in the comments after each line starting with `# BONUS TASK ADAM`.

In [14]:
import argparse
import os


def main(args):
    args.check()

    data_set = None
    if args.data_set == "mnist":
        data_set = MNIST(args.batch_size)
    if args.data_set == "fashion":
        data_set = FashionMNIST(args.batch_size)

    optimizer = None
    if args.optimizer == "adam":
        # BONUS TASK ADAM: if you implemented Adam below (and ran that code cell), then remove or comment the next line...
        # raise NotImplementedError("Adam not implemented yet.")
        # ... and uncomment the following line:
        optimizer = Adam(args.learning_rate)
    if args.optimizer == "sgd":
        optimizer = SGD(args.learning_rate)

    model = ActivatedMLP()

    train_loop(args, data_set, model, optimizer)

class TrainOptions:
    # do not change these values, rather set other values below before calling the main function
    num_epochs      = 5       # number of epochs
    batch_size      = 64      # batch size
    optimizer       = "sgd"   # adam or sgd
    data_set        = "mnist" # which dataset we want to train for
    learning_rate   = 0.1     # learning rate
    train_verbosity = 50      # how often to print the training accuracy

    def check(self):
        assert self.optimizer in ["adam", "sgd"]
        assert self.data_set in ["mnist", "fashion"]

options = TrainOptions()

# BONUS TASK ADAM: uncomment the following lines if you have implemented adam and you should be able to reach 99% test accuracy on MNIST after one or two epochs
#options.optimizer="adam"
#options.learning_rate = 0.001
# BONUS TASK ADAM: after you trained a model on MNIST, you can change the dataset to FashionMNIST and train another MLP to answer our graded bonus quiz
#options.data_set = "fashion"

main(options)
print("Training finished.")

# What now?

Now you should have implemented a few basic functions of a neural network from scratch.
We hope this has provided you with a deeper knowledge of how neural network layers are implemented and how the models can be constructed and trained.

You can now experiment a bit more with the options given above, for example:

* try to choose a different learning rate and see how it affects the training and testing accuracy
* train a model for the more challenging FashionMNIST dataset
* train for a few more epochs and see if you can achieve a higher accuracy
* try a different learning rate update rule in the training loop implementation

Make sure you attempt to complete all four tasks to the best of your abilities, as we will have questions in the graded quiz which relate to this exercise.

You can also attempt our bonus task below.

# Optional Bonus Task: Adam

In this task (which is completely optional and a bit more challenging - but at the same time highly rewarding if you manage to complete it) you can implement the famous Adam optimizer yourself.
This optimizer was first proposed in an [arxiv preprint](https://arxiv.org/abs/1412.6980) in 2014.

It is popular because it is less sensitive to choosing a suitable learning rate, and many improvements to the original implementation have since been found.
You can absolutely implement the optimizer based on the original paper linked above, but of course you are welcome to find additional sources and explanations online if you encounter difficulties.

## Mathematics and Programming

To highlight the mathematical side of deep learning we present a possible mathematical reformulation when updating the biased first moment estimates in the following.
The original paper suggests the formula:

$$m_t \leftarrow \beta_1\cdot m_{t-1} + (1-\beta_1) \cdot g_t$$

However we can simplify the equation on the right side by adding a "zero" ($m_{t-1} - m_{t-1}=0$)
$$m_t \leftarrow m_{t-1} - m_{t-1} + \beta_1 \cdot m_{t-1} + (1-\beta_1) \cdot g_t$$
This allows us to factor out $-m_{t-1}$
$$m_t \leftarrow m_{t-1} - m_{t-1} \cdot (1-\beta_1) + (1-\beta_1) \cdot g_t$$
and also factor out $(1-\beta_1)$:
$$m_t \leftarrow m_{t-1} + (1-\beta_1) \cdot (g_t - m_{t-1})$$

This may not look simpler until we realize this allows us to use the `+=` operator in Python pseudocode:
```
m += (1 - self.beta1) * (g - m)
```
Similarly the second raw moment estimate update can be reformulated to

$$v_t \leftarrow v_{t-1} + (1-\beta_2) \cdot (g_t^2 - v_{t-1})$$

which in Pseudocode can be written as `v += (1 - self.beta2) * (g * g - v)`.


In [15]:
class Adam(Optimizer):
    """
    The Adam optimizer (see https://arxiv.org/abs/1412.6980)
    :param learning_rate: initial step size
    :param beta1: Exponential decay rate of the first order moment
    :param beta2: Exponential decay rate of the second order moment
    :param eps: Small value for numerical stability
    """

    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps

        # map from layer id to a list of numpy arrays
        self.m_values = {}
        self.v_values = {}
        # map from layer id to int (time step)
        self.t_values = {}

        self.current_id = -1
        self._initialized = set()

    def run_update_rule(self, gradients, layer):
        
        self.current_id = id(layer)

        if not self.initialized:
            self.initialize(gradients)
            
        # TODO: Implement Adam Update Rule, save result in variable param_deltas and return
        # (for this bonus task you need to figure out the implementation details of Adam for yourself)
        # hint: self.m_values contains a *list* of "m_{t-1}" arrays (exactly one value with the correct shape for each gradient in the `gradients` list)
        alpha = self.learning_rate
        beta1 = self.beta1
        beta2 = self.beta2
        eps = self.eps

        t = self.t_values[self.current_id] + 1
        m_values = self.m_values[self.current_id]
        v_values = self.v_values[self.current_id]

        m_values_t_1 = []
        v_values_t_1 = []
        param_deltas = []

        for m, v, g in zip(m_values, v_values, gradients):
            m += (1 - beta1)*(g - m)
            v += (1 - beta2)*(g * g - v)

            m_corrected = m/(1 - beta1**t)
            v_corrected = v/(1 - beta2**t)

            m_values_t_1.append(m)
            v_values_t_1.append(v)
            param_deltas.append( alpha * m_corrected / (np.sqrt(v_corrected) + eps))

        self.m_values[self.current_id] = m_values_t_1
        self.v_values[self.current_id] = v_values_t_1
        self.t_values[self.current_id] = t

        return param_deltas
        
    def initialize(self, gradients):
        self.m_values[self.current_id] = [np.zeros_like(gradient) for gradient in gradients]
        self.v_values[self.current_id] = [np.zeros_like(gradient) for gradient in gradients]
        self.t_values[self.current_id] = 0
        self._initialized.add(self.current_id)

    @property
    def initialized(self):
        return self.current_id in self._initialized

If your Adam implemenation is correct you should be able to achieve about .99 **test** accuracy on **MNIST** after the **first** or **second** epoch (with a learning rate of 0.001).
Edit the code cell in the section "Running the Training" to test this (look for the lines with comments containing `# BONUS TASK ADAM`).

If your implementation is correct you can gain 2 bonus points for completing our [bonus quiz](https://learn.ki-campus.org/courses/edgeai-hpi2022/items/7GjtyHsBaGNtWNVbbvPqxa) of this week.
Before you attempt the quiz, you should train an MLP on **FashionMNIST** with Adam with the same settings (you need to uncomment another line in the training code to change the dataset - keep the learning rate of 0.001).

### Testing Adam implementation

In [16]:
options = TrainOptions()
options.optimizer="adam"
options.learning_rate = 0.001

print('Training on MNIST')
main(options)
print('Training finished.')

print('\n\n')
print('Training on Fashion')
options.data_set = "fashion"
main(options)
print('Training finished.')

## The four proposed tasks

### PT1. Changing the learning rate

We will verify the influence of the learning rate on the model with SGD optimizer by changing its value and observing the accuracy of the model during the training process. The choosen values for the learning rate were 10.0, 5.0, 1.0, 0.01, 0.001, and 0.0001. 

In [17]:
for lr in [10.0, 5.0, 1.0, 0.01, 0.001, 0.0001]:
    print('Learning rate: {}'.format(lr))
    print('\n')
    options = TrainOptions()
    
    options.batchsize = 128
    options.learning_rate = lr

    main(options)
    print('Training finished')
    print('\n\n\n')

As we already know, the learning rate determines the step size that the optimizer is allowed to take in the direction given by the gradient vector. Given the results above, one may observe that, in this case, the learning rate may not be so high nor so small. Having the learning rate as big as 10 made the process take 2 epochs to achieve a precision of 0.8 (in this case, I thought it would diverge). On the other hand, reducing the learning rate resulted in a slower convergence process. It became so slow in the last three cases, that the model didn't achieved an accuracy higher than 0.7 - the last one achieved a poor accuracy of 0.15. In fact, it barely moved from the accuracy obtained in the first epoch. 

For two values, 5.0 and 1.0, the accuracy fastly converges. In the first epoch, it already achieves values equal or higher than 0.95, for both training and testing. It means that in just 900 iteractions, we already have a great model.

### PT2. Train a model for the FashionMNIST dataset

In [18]:
options = TrainOptions()
options.dataset = 'fashion'

main(options)
print('Training finished')

### PT3. Train for a few more epochs and see if you can achieve a higher accuracy


In [19]:
options = TrainOptions()
options.num_epochs = 10
main(options)
print('Training finished.')

### PT4. Try a different learning rate update rule in the training loop implementation

In [20]:
def train_loop_nuplr(args, data_set, model, optimizer):
    # Training loop with new update rule for learning rate
    for epoch in range(args.num_epochs):
        # train our model
        for iteration, batch in enumerate(data_set.train):
            model.forward(batch)
            model.backward(optimizer)

            if iteration % args.train_verbosity == 0:
                accuracy = F.accuracy(model.predictions, batch.labels).data
                print("train: epoch: {: 2d}, loss: {: 5.2f}, accuracy {:.2f}, iteration: {: 4d}".
                      format(epoch, model.loss.data, accuracy, iteration), end="\r")
        
        # test our model
        print("\nrunning test set...")
        sum_accuracy = 0.0
        sum_loss = 0.0
        for iterations, batch in enumerate(data_set.test):
            model.forward(batch, train=False)
            sum_accuracy += F.accuracy(model.predictions, batch.labels).data
            sum_loss += model.loss.data
        nr_batches = iterations - 1
        print(" test: epoch: {: 2d}, loss: {: 5.2f}, accuracy {:.2f}".
              format(epoch, sum_loss / nr_batches, sum_accuracy / nr_batches))
        
        # Here we change the update rule for the learning rate. In this case, we just set the epoch when the lr 
        # is reduced to 10% of the previous value.
        if epoch % 5 == 0:
            optimizer.learning_rate *= 0.1

            
def main_new_update_rule_lr(args):
    args.check()

    data_set = None
    if args.data_set == "mnist":
        data_set = MNIST(args.batch_size)
    if args.data_set == "fashion":
        data_set = FashionMNIST(args.batch_size)

    optimizer = None
    if args.optimizer == "adam":
        # BONUS TASK ADAM: if you implemented Adam below (and ran that code cell), then remove or comment the next line...
        raise NotImplementedError("Adam not implemented yet.")
        # ... and uncomment the following line:
        #optimizer = Adam(args.learning_rate)
    if args.optimizer == "sgd":
        optimizer = SGD(args.learning_rate)

    model = ActivatedMLP()

    train_loop_nuplr(args, data_set, model, optimizer)

In [21]:
options = TrainOptions()
options.num_epochs = 10
main_new_update_rule_lr(options)
print('Training finished.')

As one may observe, the update rule has an impact on the model obtained. Since the update rule is a way to balance the advantages and disadvantages of having a large (or little) learning rate, it expected that changing this rule reflects on the convergence process of training the model. In the case above, were we just changed the epoch when the lr is reduced, we see that the final accuracy is slighlty higher than the one obtained in the **PT3**.